In [2]:
!pip install git+https://github.com/firekind/athena

  Cloning https://github.com/firekind/athena to /tmp/pip-req-build-nxvd7owd
  Running command git clone -q https://github.com/firekind/athena /tmp/pip-req-build-nxvd7owd
  Created wheel for athena: filename=athena-1.0.0-cp36-none-any.whl size=38568 sha256=2dfd86481e0ac976743a5f0dad3e0ee6eb46185b9ba39f0f7d43cfbcbfb5fce9
  Stored in directory: /tmp/pip-ephem-wheel-cache-111f7ns1/wheels/36/d3/7b/d9500a9af2498abc5d48d60c7db37c1456ee745049da352e0b
Successfully built athena


In [3]:
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, OneCycleLR

from athena import datasets, Experiment, Experiments, ClassificationSolver
# from athena.utils.transforms import ToNumpy, ToTensor
from athena.models import ResNet18

In [4]:
# from athena.utils.functions import plot_experiments
from athena.utils.transforms import ToNumpy   #, ToTensor

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

batch_size = 128 if device == 'cuda' else 64
epochs = 100

cuda


In [7]:
net = ResNet18().to(device)

summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [8]:
def custom_loss_fn(y_pred, y_true):
   y_pred = F.log_softmax(y_pred, dim=-1)
   return F.nll_loss(y_pred, y_true)

In [9]:
# creating the experiment
exp = (
    Experiment("New Resnet 18 Model")
    .model(ResNet18())
    .solver(ClassificationSolver)
        .optimizer(optim.SGD, lr=0.01, momentum=0.9)
        .scheduler(StepLR, step_size=8, gamma=0.1)
        .epochs(epochs)
        .train_loader(
            datasets.cifar10()
            .batch_size(batch_size)
            .train()
            .transform(A.Compose([
                                  A.Lambda(ToNumpy),
                                  A.HorizontalFlip(p=0.5),
                                  A.RandomBrightnessContrast(p=0.2),
                                  A.HueSaturationValue(),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                                  ToTensor()
              ]))
            .build() 
          )
        .test_loader(
            datasets.cifar10()
            .batch_size(batch_size)
            .test()
            .transform(A.Compose([
                              A.Lambda(ToNumpy),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                              ToTensor(),
                          ]))
            .build())
        .loss_fn(custom_loss_fn)
        .device("cuda")
        .build()
     .build() 
)

# running experiment
exp.run()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
=> Running experiment: New Resnet 18 Model
Epoch: 1 / 100
782/782 [==============================] - 29s 37ms/step - train loss: 1.3486 - train accuracy: 51.5760
Test set: Average loss: 1.0346, Accuracy: 6476/10000 (64.76%)

Epoch: 2 / 100
782/782 [==============================] - 29s 37ms/step - train loss: 0.7980 - train accuracy: 72.1740
Test set: Average loss: 0.8093, Accuracy: 7225/10000 (72.25%)

Epoch: 3 / 100
782/782 [==============================] - 29s 37ms/step - train loss: 0.6010 - train accuracy: 79.0800
Test set: Average loss: 0.7155, Accuracy: 7643/10000 (76.43%)

Epoch: 4 / 100
782/782 [==============================] - 29s 37ms/step - train loss: 0.4870 - train accuracy: 83.2600
Test set: Average loss: 0.5320, Accuracy: 8256/10000 (82.56%)

Epoch: 5 / 100
782/782 [==============================] - 29s 37ms/step - train loss: 0.4164 - train accuracy: 85.4900
Test set: Average los